In [3]:
from bs4 import BeautifulSoup
import requests
from lxml import etree 
from lxml import html

In [6]:
def diff_html(elem1, elem2, path=[]):
    result = []
    if elem1.text != elem2.text and elem1.tag != 'script':
#         print('/'.join(path))
        result.append('/'.join(path))
#         print( "Diff: [%s ] != [ %s ] " % (elem1.text, elem2.text) )
    for idx1, child1 in enumerate(elem1.getchildren()):
        if not isinstance(child1, html.HtmlComment):
            found_child2 = False
            for child2 in elem2[ idx1:]:
                if child1.tag == child2.tag and child1.attrib == child2.attrib:
                    tag = child1.tag
#                     if 'class' in child1.attrib:
#                         tag += '[@class="%s"]' % child1.attrib['class']
                    result += diff_html(child1, child2, path + [ tag ])
                    found_child2 = True
                    break
            if not found_child2:
                for child2 in elem2[ idx1:]:
                    if child1.tag == child2.tag:
                        tag = child1.tag
#                         if 'class' in child1.attrib:
#                             tag += '[@class="%s"]' % child1.attrib['class']
                        result += diff_html(child1, child2, path + [ tag ])
                        break
    return result

In [261]:
def print_content(tree):
    index = 0
    for child in tree.getchildren():
        print( '-- INDEX --- %s' % index)
        if not isinstance(child, html.HtmlComment):
            print(child.text_content())
        index += 1

In [18]:
# url1 = 'https://www.bbc.com/vietnamese/culture-social-45514193'
# url2 = 'https://www.bbc.com/vietnamese/vert-cul-45653884'
# url1 = 'https://adsplus.vn/blog/huong-da%CC%83n-cach-lam-sao-de-tang-view-tren-youtube/'
# url2 = 'https://adsplus.vn/blog/diem-qua-dich-vu-nghe-nhac-truc-tiep-youtube-music/'
# url1 = 'https://advertisingvietnam.com/2017/06/content-tips-89-cach-viet-noi-dung/'
# url2 = 'https://advertisingvietnam.com/2017/03/nhung-loi-co-ban-thuong-gap-trong-copywriting/'
# url1 = 'https://kyna.vn/bai-viet/hoc-seo-o-dau-tot-nhat-tphcm-dia-chi-tin-cay-nhat-cho-ban.html'
# url2 = 'https://kyna.vn/bai-viet/co-the-ban-chua-biet-hoc-seo-can-kien-thuc-gi.html'
# url1 = 'http://www.brandsvietnam.com/congdong/topic/12673-7-ly-do-tai-sao-nen-su-dung-Landing-page-cho-Quang-cao'
# url2 = 'http://www.brandsvietnam.com/16272-Nghe-tao-anh-huong'
# url1 = 'http://vnetcom.vn/blogs/Kinh-Nghiem-Chon-Lua-doi-Tac-Thiet-Ke-Website-Uy-Tin-.html'
# url2 = 'http://vnetcom.vn/blogs/Digital-marketing-%E2%80%93-6-du-bao-cho-nam-2015.html'
# url1 = 'https://www.butpha.com/huong-dan-lam-influencer-marketing'
# url2 = 'https://www.butpha.com/lo-trinh-chien-dich-influencer'
url1 = 'http://vietnamnet.vn/vn/thoi-su/may-bay-roi-o-tam-dao-10-gio-bang-rung-tim-kiem-hai-cot-2-phi-cong-480584.html'
url2 = 'http://vietnamnet.vn/vn/thoi-su/bao-ke-cho-long-bien-dinh-chi-pho-ban-quan-ly-dung-2-doi-boc-xep-480622.html'

In [19]:
tree1 = html.fromstring(requests.get(url1).content)
tree2 = html.fromstring(requests.get(url2).content)


In [20]:
res = diff_html(tree1, tree2)
set(res)

{'body/div/div/div/div/div',
 'body/div/div/div/div/div/div/div/div/a',
 'body/div/div/div/div/div/div/div/div/div/ul/li/h3/a',
 'body/div/div/div/div/div/div/div/p',
 'body/div/div/div/div/div/div/div/p/a',
 'body/div/div/div/div/div/div/div/span',
 'body/div/div/div/div/div/div/div/table/tbody/tr/td',
 'body/div/div/div/div/div/div/h1',
 'body/div/div/div/div/div/div/ul/li/h3/a',
 'body/div/div/div/div/div/div/ul/li/h4/a',
 'body/div/div/div/div/div/div/ul/li/h5/a',
 'head/title'}

In [21]:

for res_xpath in set(res):
    print('-----------%s--------------' % res_xpath)
    values = []
    for elem in tree1.xpath(res_xpath):
        if elem.text and elem.text.strip():
            values.append(elem.text_content())
    print('\n'.join(values))
#         print(etree.tostring(elem, encoding='unicode'))


-----------body/div/div/div/div/div/div/ul/li/h4/a--------------
Bảo kê chợ Long Biên: Đình chỉ phó ban quản lý, dừng 2 đội bốc xếp
Giá vàng hôm nay 1/10: USD tăng cao, chẳng ai dám mua vàng
Đường 500 tỷ đình trệ, máy móc vùi trong núi lở
Bố mẹ từ chối điều trị ung thư, quyết ôm con về khiến bác sĩ bất lực
Chủ nhân giàu có của phòng trà ca nhạc nổi tiếng Hà thành xưa
Thứ phế thải bỏ đi chế thành đặc sản: Món lừng danh Thái Lan, Việt Nam
TP.HCM: Công bố sai phạm hơn 2.000ha đất công tại Sagri
Giật mình bầy sâu trên đĩa mồi nhậu thơm nức của dân Việt
Chi 12 tỷ xây nhà hỏa táng cho cỏ mọc um tùm
-----------body/div/div/div/div/div/div/div/p--------------
Đại tá Ngô Hồng Thái, Chính ủy Bộ chỉ huy quân sự tỉnh Thái Nguyên, Phó Ban chỉ đạo 515 - Ban tìm kiếm được UBND tỉnh thành lập từ tháng 8/2018, trực tiếp tham gia cuộc tìm kiếm chiếc Mig-21U rơi trên đỉnh núi Tam Đảo.
Theo Đại tá Ngô Hồng Thái, cuộc tìm kiếm được chia thành nhiều giai đoạn, lực lượng tham gia mỗi đợt lên tới gần 20 người

In [464]:
from basestring import unicode

ModuleNotFoundError: No module named 'basestring'